# Step 8: Warp the moving image

Note that this is a CPU-based warping... a faster GPU-based warping using pytorch is in `torch_reg.py`

In [1]:
import os
import zarr
from precomputed_tif.zarr_stack import ZarrStack
from phathom import io
import phathom.io.conversion as conversion
from phathom.registration import registration
from phathom.utils import pickle_load

In [2]:
working_dir = '/media/jswaney/Drive/Justin/marmoset/'

In [3]:
# Open images
fixed_zarr_path = 'round1/syto16.zarr/1_1_1'
moving_zarr_path = 'round2/syto16.zarr/1_1_1'

fixed_img = io.zarr.open(os.path.join(working_dir, fixed_zarr_path), 
                         mode='r')
moving_img = io.zarr.open(os.path.join(working_dir, moving_zarr_path), 
                          mode='r')

fixed_img.shape

(3600, 9020, 7275)

In [4]:
# Load the coordinate interpolator
interpolator_path = 'map_interpolator.pkl'

interpolator = pickle_load(os.path.join(working_dir, 
                                        interpolator_path))

In [5]:
# Open zarr array for the registered image
nonrigid_zarr_path = 'round2/registered/1_1_1'

nonrigid_img = io.zarr.new_zarr(os.path.join(working_dir, 
                                             nonrigid_zarr_path),
                                fixed_img.shape,
                                fixed_img.chunks,
                                fixed_img.dtype)

In [6]:
# Warp the entire moving image
nb_workers = 1
batch_size = None
padding = 4

registration.register(moving_img, 
                      nonrigid_img, 
                      os.path.join(working_dir, interpolator_path), 
                      nb_workers, 
                      batch_size=batch_size, 
                      padding=padding)

TypeError: register() missing 1 required positional argument: 'nb_workers'

In [4]:
# Make the mipmap stack for neuroglancer
n_levels = 5
reg_dest_path = 'tubb3_zarr'
reg_src_path = 'tubb3_zarr/1_1_1'

reg_src = zarr.NestedDirectoryStore(os.path.join(working_dir, 
                                                 reg_src_path))
reg_dest = os.path.join(working_dir, reg_dest_path)
reg_stack = ZarrStack(reg_src, reg_dest)

# Write info and downsampled mipmaps
reg_stack.write_info_file(n_levels)
for level in range(2, n_levels+1):
    reg_stack.write_level_n(level)

100%|██████████| 9/9 [00:00<00:00, 23.14it/s]


In [4]:
# Write out to individual tiffs
zarr_path = 'fixed/zarr_stack/2_2_2'
output_dir = 'fixed/tiffs_2x'
nb_workers = 48
compress = 1

zarr_store = zarr.NestedDirectoryStore(os.path.join(working_dir, zarr_path))

conversion.zarr_to_tifs(zarr_store, 
                        os.path.join(working_dir, output_dir),
                        nb_workers=nb_workers, 
                        compress=compress)